In [7]:
import os
import json

result_dir = "../exp_results"
num_cores = [1]
num_tasks = [3,6,9,12]
utilizations = [0.4, 0.6, 0.8]
num_tasksets = 20
summary_path = result_dir + "/summary.csv"

# if summary file exists, remove it
if os.path.exists(summary_path):
    os.remove(summary_path)
    
header = "numCores,numTasks,utilization,tasksetIndex,realLinux_schedulability"
with open(summary_path, "a") as f:
    f.write(header + "\n")

for num_core in num_cores:
    for num_task in num_tasks:
        for utilization in utilizations:
            for i in range(num_tasksets):
                result_file = result_dir + "/details/" + str(num_core) + "cores/" + str(num_task) + "tasks/" + str(utilization) + "utilization/result_" + str(i) + ".json"
                with open(result_file, 'r') as file:
                    tasks_result = json.load(file)
                assert len(tasks_result) == num_task
                
                realLinux_schedulability = True
                for task_result in tasks_result:
                    if task_result["wcrt_ns"] > task_result["deadline_ns"]:
                        realLinux_schedulability = False
                        break
                with open(summary_path, "a") as f:
                    f.write(str(num_core) + "," + str(num_task) + "," + str(utilization) + "," + str(i) + "," + str(realLinux_schedulability) + "\n")
                
